In [1]:
import os
import numpy as np
import pandas as pd
import streamlit as st
from pathlib import Path
from datetime import date
from utils.asset_database import DICT_ASSET_INFO
from utils.tools import compute_dividends,compute_position

import yfinance as yf

In [2]:
current_path = Path.cwd()

#### Load data ####
ARQUIVO_DADOS = current_path / "data" / "historico_transacoes.csv"

#### Import data ####
base_dados = pd.read_csv(ARQUIVO_DADOS)
base_dados["Data"] = pd.to_datetime(base_dados["Data"], format="%Y-%m-%d")




In [ ]:
# data_final = base_dados["Data"].iloc[0] + pd.Timedelta(days=1)
# data_inicial = base_dados["Data"].iloc[-1]

# currency = "EUR"
# moeda_atual = "USD"

# exchange_rate = yf.Ticker(f"EURUSD=X").history(start=data_inicial,end=data_final)[["Close"]]
# exchange_rate.index = pd.to_datetime(exchange_rate.index, )

# exchange_rate.asof(pd.Timestamp("2025-01-08",tz="UTC"))["Close"]

np.float64(1.034554123878479)

In [7]:
from copy import deepcopy
def convert_to_currency(base_dados,currency):

    data_final = base_dados["Data"].iloc[0] + pd.Timedelta(days=1)
    data_inicial = base_dados["Data"].iloc[-1]
 
    lista_moedas = ["USD","EUR","BRL"]
    lista_moedas.remove(currency)
    dic_historico_convercao = {}
    for moeda_inicial in lista_moedas:

        historico = yf.Ticker(f"{moeda_inicial}{currency}=X").history(start=data_inicial,end=data_final)[["Close"]]
        dic_historico_convercao.update({f"{moeda_inicial}_to_{currency}":historico})

    df = deepcopy(base_dados)
    def conversion_function(row):
        if row["Moeda"] != currency:
            moeda_atual = row["Moeda"]
            data = row["Data"]

            exchange_rate = dic_historico_convercao[
                f"{moeda_atual}_to_{currency}"
            ].asof(pd.Timestamp(data,tz="UTC"))["Close"]
            
        else:
            exchange_rate = 1
        return np.round(row["Preço"] * exchange_rate,2)
    df["Preço"] = df.apply(conversion_function, axis=1)
    df["Moeda"] = currency
    return df

    
base_dados_BRL = convert_to_currency(base_dados,"BRL")
base_dados_USD = convert_to_currency(base_dados,"USD")
base_dados_EUR = convert_to_currency(base_dados,"EUR")


In [8]:
display(
base_dados_BRL[base_dados_BRL["Ativo"] == "TLT"],
base_dados_USD[base_dados_USD["Ativo"] == "TLT"],
base_dados_EUR[base_dados_EUR["Ativo"] == "TLT"]
)


,Dono,Data,Ativo,C/V,Preço,Qnt,Moeda
17,Luiz,2025-04-03,TLT,C,490.25,6,BRL
32,Luiz,2025-03-13,TLT,C,520.76,2,BRL
73,Luiz,2024-11-29,TLT,C,562.60,6,BRL
141,Luiz,2024-06-18,TLT,C,508.99,1,BRL
220,Luiz,2024-03-13,TLT,C,468.87,2,BRL
468,Luiz,2023-11-28,TLT,C,445.91,1,BRL
469,Luiz,2023-11-22,TLT,C,445.67,2,BRL
495,Luiz,2023-10-04,TLT,C,442.91,1,BRL
512,Luiz,2023-08-28,TLT,C,467.96,1,BRL


,Dono,Data,Ativo,C/V,Preço,Qnt,Moeda
17,Luiz,2025-04-03,TLT,C,86.56,6,USD
32,Luiz,2025-03-13,TLT,C,89.82,2,USD
73,Luiz,2024-11-29,TLT,C,93.55,6,USD
141,Luiz,2024-06-18,TLT,C,93.84,1,USD
220,Luiz,2024-03-13,TLT,C,94.36,2,USD
468,Luiz,2023-11-28,TLT,C,91.09,1,USD
469,Luiz,2023-11-22,TLT,C,90.97,2,USD
495,Luiz,2023-10-04,TLT,C,85.73,1,USD
512,Luiz,2023-08-28,TLT,C,96.08,1,USD


,Dono,Data,Ativo,C/V,Preço,Qnt,Moeda
17,Luiz,2025-04-03,TLT,C,79.35,6,EUR
32,Luiz,2025-03-13,TLT,C,82.51,2,EUR
73,Luiz,2024-11-29,TLT,C,88.56,6,EUR
141,Luiz,2024-06-18,TLT,C,87.24,1,EUR
220,Luiz,2024-03-13,TLT,C,86.27,2,EUR
468,Luiz,2023-11-28,TLT,C,83.03,1,EUR
469,Luiz,2023-11-22,TLT,C,83.25,2,EUR
495,Luiz,2023-10-04,TLT,C,81.81,1,EUR
512,Luiz,2023-08-28,TLT,C,88.91,1,EUR


In [5]:
dividens = compute_dividends(base_dados)

$CIEL3.SA: possibly delisted; no price data found  (1d 1926-05-15 -> 2025-04-20) (Yahoo error = "No data found, symbol may be delisted")


In [6]:
dividens["Data"] = pd.to_datetime(dividens["Data"], format="%Y-%m-%d", utc=True)
dividens["Data"] = pd.to_datetime(dividens["Data"]).dt.date

In [7]:
dividens

,Data,Ativo,Classe,Dividendo,Qnt,Dividendo total
0,2023-06-01,JSRE11,FII,0.460000,7,3.220000
1,2023-06-01,HSML11,FII,0.737854,6,4.427124
2,2023-06-01,BCRI11,FII,0.800000,38,30.400000
3,2023-06-02,BBAS3,Ação,0.061504,162,9.963648
4,2023-06-02,GGRC11,FII,0.104393,27,2.818611
...,...,...,...,...,...,...
357,2025-04-01,TLT,USD,1.880000,16,30.030000
358,2025-04-01,BIL,USD,1.850000,18,33.270000
359,2025-04-09,CLIN11,FII,1.100000,10,11.000000
360,2025-04-15,AFHI11,FII,0.990000,6,5.940000


In [8]:
ano_selecionado = 2025
mes_selecionado = 3

dividendos_mensais = dividens[
    (dividens["Data"].apply(lambda value: value.year) == ano_selecionado) &
    (dividens["Data"].apply(lambda value: value.month) == mes_selecionado)
].drop(["Data"],axis=1)

dividendos_agrupados = pd.DataFrame(
    columns=dividendos_mensais["Classe"].unique(),
    data=[[dividendos_mensais[dividendos_mensais["Classe"] == classe]["Dividendo total"].sum() for classe in dividendos_mensais["Classe"].unique()]]
)

dividendos_agrupados["Total"] = dividendos_agrupados.apply(sum,axis=1)
# dividendo_agrupado

In [ ]:
position = compute_position(base_dados[base_dados["Dono"] == "Luiz"],pd.Timestamp(year=ano_selecionado,month=mes_selecionado,day=1))

In [ ]:
movimentacao = base_dados[
    (base_dados["Data"].apply(lambda value: value.year) == ano_selecionado) &
    (base_dados["Data"].apply(lambda value: value.month) == mes_selecionado) &
    (base_dados["Dono"] == "Luiz")
].drop(["Dono"],axis=1)

compras = movimentacao[movimentacao["C/V"] == "C"]
vendas = movimentacao[movimentacao["C/V"] == "V"]

# mask_compra = position["Ativo"].apply(lambda value: value in compras["Ativo"].unique())
# mask_venda = position["Ativo"].apply(lambda value: value in vendas["Ativo"].unique())

In [ ]:



# np.array([position[position['Ativo'] == ativo]["Custo unitário"].values for ativo in compras["Ativo"]])

# np.array([position[position['Ativo'] == ativo]["Qnt"].values for ativo in compras["Ativo"]])

array([ 24.82277778, 109.55944444,  73.73      ,   0.        ,
         0.        , 501.9651817 , 189.44396993, 104.98      ,
       112.66555556, 128.98      ,  78.34      ,   0.        ,
         0.        ,   0.        ])

In [18]:
display(
    position[mask_compra],
    compras,
)

,Ativo,Classe,Qnt,Custo unitário,Preço unitário,Custo total,Valor total
29,TRPL4,Ação,36,24.822778,22.54,893.62,811.44
26,TRXF11,FII,9,109.559444,100.63,986.03,905.67
12,HGRE11,FII,9,112.665556,100.36,1013.99,903.24
30,HGCR11,FII,9,104.980000,94.65,944.82,851.85
21,HGRU11,FII,7,128.980000,112.92,902.86,790.44
6,PVBI11,FII,7,73.730000,76.18,516.11,533.26
3,HSLG11,FII,7,78.340000,71.12,548.38,497.84
9,TLT,USD,14,501.965182,536.08,7027.51,7505.12
13,IBIT,USD,25,189.443970,279.69,4736.10,6992.25


,Data,Ativo,C/V,Preço,Qnt,Moeda,Classe
23,2025-03-25,TRPL4,C,22.400,23,BRL,Ação
24,2025-03-25,TRXF11,C,99.880,2,BRL,FII
25,2025-03-25,PVBI11,C,76.670,7,BRL,FII
26,2025-03-25,IPCA+ 2050,C,7.740,45,BRL,Tesouro
28,2025-03-21,TSLA,C,241.630,1,USD,USD
32,2025-03-13,TLT,C,89.820,2,USD,USD
33,2025-03-13,IBIT,C,46.380,5,USD,USD
34,2025-03-12,HGCR11,C,94.675,4,BRL,FII
35,2025-03-12,HGRE11,C,100.040,2,BRL,FII
36,2025-03-12,HGRU11,C,115.000,2,BRL,FII


In [36]:
resultado_compra = pd.DataFrame({
    "Ativo":compras["Ativo"],
    "Preço médio":get_field_for_assets_in_position(position,compras["Ativo"],"Preço unitário"),
    "Qnt atual":get_field_for_assets_in_position(position,compras["Ativo"],"Qnt"),
    "Valor atual":get_field_for_assets_in_position(position,compras["Ativo"],"Preço unitário") * get_field_for_assets_in_position(position,compras["Ativo"],"Qnt"),
    "Preço compra":compras["Preço"],
    "Qtn compra":compras["Qnt"],
    "Valor comprado":compras["Preço"]*compras["Qnt"],
})

In [38]:
resultado_compra

,Ativo,Preço médio,Qnt atual,Valor atual,Preço compra,Qtn compra,Valor comprado
23,TRPL4,22.54,36,811.44,22.400,23,515.20
24,TRXF11,100.63,9,905.67,99.880,2,199.76
25,PVBI11,76.18,7,533.26,76.670,7,536.69
26,IPCA+ 2050,0.00,0,0.00,7.740,45,348.30
28,TSLA,0.00,0,0.00,241.630,1,241.63
32,TLT,536.08,14,7505.12,89.820,2,179.64
33,IBIT,279.69,25,6992.25,46.380,5,231.90
34,HGCR11,94.65,9,851.85,94.675,4,378.70
35,HGRE11,100.36,9,903.24,100.040,2,200.08
36,HGRU11,112.92,7,790.44,115.000,2,230.00


In [39]:
resultado_venda = pd.DataFrame({
    "Ativo":vendas["Ativo"],
    "Preço médio":get_field_for_assets_in_position(position,vendas["Ativo"],"Preço unitário"),
    "Qnt atual":get_field_for_assets_in_position(position,vendas["Ativo"],"Qnt"),
    "Valor atual":get_field_for_assets_in_position(position,vendas["Ativo"],"Preço unitário") * get_field_for_assets_in_position(position,vendas["Ativo"],"Qnt"),
    "Preço venda":vendas["Preço"],
    "Qtn venda":vendas["Qnt"],
    "Valor vendido":vendas["Preço"]*vendas["Qnt"],
})

resultado_venda["Lucro"] = resultado_venda.apply(lambda row: (row["Preço venda"]-row["Preço médio"]) * row["Qtn venda"],axis=1)
resultado_venda

,Ativo,Preço médio,Qnt atual,Valor atual,Preço venda,Qtn venda,Valor vendido,Lucro
29,BIL,532.0,25,13300.0,91.62,3,274.86,-1321.14
39,KNSC11,8.4,106,890.4,8.58,106,909.48,19.08
42,BIL,532.0,25,13300.0,91.53,4,366.12,-1761.88
